# $\sqrt{i\text{SWAP}}$ gate in superconducting qubits

A transmon is a Josephson junction (non linear inductor) shunted with a capacitor. The quantum mechanical Hamiltonian of such a system often referred to as a Cooper pair box is given by \cite{Morgado2021, Jones2021},
    
$$ H = 4 E_c (n - n_g)^2 - E_J \cos\varphi,$$
where $n$ and $\varphi$ are the Cooper pair number and phase conjugate variables, $n_g$ is charge bias, $E_J$ and $E_c$ denote the Josephson and charging energies respectively.  The non linear nature of the Josephson junction makes the transmon an anharmonic oscillator, effectively isolating the first two levels from the remaining levels to realize a qubit (i.e. $\omega_{01}\neq \omega_{12}$). The evolution of transmons is modelled with a generalized Duffing Oscillator resulting from the expansion of Cooper pair box Hamiltonian,
    
$$H_{DO_K} = \omega_0 \left(a^{\dagger}a + \frac{1}{2}\right) +\frac{\omega_0^2}{8\eta} \sum_{k=2}^{K} \left(\frac{4\eta}{\omega_0}\right)^{k}\frac{(-1)^k (a^{\dagger} + a)^{2k}}{(2k)!} +  i f(t)\cos[\omega'(t)t + \phi(t)](a - a^{\dagger}), $$
where $\omega_0$ is the qubit frequency, $\eta$ is the anharmonicity (note that $\omega_{01} \approx \omega_0 + \eta $), $f(t)$ is the slow varying envelope of the RF drive. The microwave frequency $\omega'$ is typically in the $GHz$ range and therefore it is generated by mixing a low frequency signal of few $MHz$ with a local oscillator $\omega' = \omega_{LO} + \omega_{IF}$. The $I$ and $Q$ quadratures of slow varying envelope,
    
$$[I(t), Q(t)] = [f(t)\cos(\phi(t)), f(t) \sin(\phi(t))].$$
    
Jones \textit{et al.} \cite{Jones2021} argue that for typical transmon parameters, sextic terms ($K=3$) are enough to match the spectrum of the Cooper pair box Hamiltonian ($K=\infty$). In general, the minimum number of levels and expansion order needed to accurately model the dynamics depends on the anharmonicity, strength of the drive etc. these parameters have to be chosen based on the desired accuracy, for e.g. by monitoring the occupation probabilities of lower energy levels.

In [1]:
using Revise
using QuantumOptimalControl
using QuantumOptics
using LinearAlgebra
using Flux, DiffEqFlux
using Optim
using PlotlyJS
using DifferentialEquations: DP5, Tsit5, Vern7, Vern9, BS3
using Random
using ProgressMeter
ProgressMeter.ijulia_behavior(:clear)

WebIO._IJuliaInit()

┌ Info: Precompiling QuantumOptimalControl [e91afccf-c93e-44fd-aa4c-3d3ef13645c0]
└ @ Base loading.jl:1423


false

In [2]:
ω₁ = 2π*5.0
η₁ = -2π*300.0*1e-3
ω₁ += η₁

ω₂ = 2π*5.0

η₂ = -2π*300.0*1e-3
ω₂ += η₂

ωlo1 = ω₂
ωlo2 = ω₁

29.530970943744055

In [3]:
n_levels = 6
bs = FockBasis(n_levels-1)
id = identityoperator(bs)

a1 = destroy(bs)⊗id
a1d = create(bs)⊗id
a2 = id⊗destroy(bs)
a2d = id⊗create(bs)
id12 = id⊗id;

In [4]:
H0 = (η₁/12.0)*(a1 + a1d)^4 + (η₂/12.0)*(a2 + a2d)^4 ## idle qubit, real
H1 = (a2 - a2d)*(a1 - a1d) ## qubit-qubit
H2 = a1d*a1 + 0.5*id12
H3 = a2d*a2 + 0.5*id12
t0, t1 = 0.0, 16.0

(0.0, 16.0)

In [5]:
Random.seed!(3)
n_neurons = 8
ann = FastChain(FastDense(1, n_neurons, tanh), 
                FastDense(n_neurons, n_neurons, tanh), 
                FastDense(n_neurons, n_neurons, tanh), 
                FastDense(n_neurons, 3))
θ = Vector{Float64}(initial_params(ann))     
n_params = length(θ)

187

In [6]:
g_guess(t) = @. 0.01*2π*exp(-(t-0.5*t1)^2/(0.2*t1)^2)
f1_guess(t) = @. 0.1*2π*exp(-(t-0.5*t1)^2/(0.2*t1)^2)
f2_guess(t) = @. 0.1*2π*exp(-(t-0.5*t1)^2/(0.2*t1)^2)

tsf32 = Float32(t0):0.1f0:Float32(t1)
gs = Vector{Float32}(g_guess(tsf32))
f1s = Vector{Float32}(f1_guess(tsf32))
f2s = Vector{Float32}(f2_guess(tsf32))

ts = Vector{Float64}(tsf32)
function loss(p)
    c = 0.0f0
    for (i,t) in enumerate(tsf32)
        x = ann([t], p)
        c += (x[1] - gs[i])^2
        c += (x[2] - f1s[i])^2
        c += (x[3] - f2s[i])^2
    end
   # println(c)
    c
end

res = DiffEqFlux.sciml_train(loss, initial_params(ann), ADAM(0.1f0), maxiters = 500)
θ = Vector{Float64}(res.u);

In [7]:
coeffs(p, t) = let v=ann([t], p)
                    [v[1], ω₁ + v[2], ω₂ + v[3]]
               end 

H = Hamiltonian(H0, [H1, H2, H3], coeffs);

In [8]:
states = [fockstate(bs, 0)⊗fockstate(bs, 0),
          fockstate(bs, 0)⊗fockstate(bs, 1),
          fockstate(bs, 1)⊗fockstate(bs, 0),
          fockstate(bs, 1)⊗fockstate(bs, 1)]

trans = UnitaryTransform(states, [[1 0 0 0];
                                  [0 1 1.0im 0]/√2;
                                  [0 1.0im 1 0]/√2;
                                  [0 0.0 0.0 1.0]]);

In [9]:
bcs(p) = 2.0*sum(ann([t0], p).^2 + ann([t1], p).^2)
                 
cost = CostFunction((x,y)-> 1.0.-real(x'*y), bcs)

CostFunction(var"#1#2"(), bcs)

In [10]:
prob = QOCProblem(H, trans,(t0, t1), cost);

In [11]:
sol = solve(prob, θ, ADAM(0.01); maxiter=100)

Progress: 100%|█████████████████████████████████████████| Time: 0:40:35
  distance:     0.008843759024369402
  constraints:  0.00032473383128927565


Solution{Float64}([0.19779067439460907, -0.20758995659703716, -0.6652511657938235, 0.21809488850260766, -0.3588405777083468, 1.0385710969109891, -0.4239301179225393, -0.3967171421543028, -1.6936378222887152, 1.3783469909029404  …  0.4065969496708377, 0.36084045504442464, -0.11231538429798696, 0.013775967215829963, -0.4728479890811861, -0.11245905660143214, -0.22466603372114477, 0.014441332888218226, 0.025673636646376223, -0.023986341394559785], [1.3543179657787427, 1.07655379073858, 0.8689863216012361, 0.8850320761549992, 0.7207416809467425, 0.2607578189158947, 0.12651999669021063, 0.17065461761975687, 0.3210239806216659, 0.2899867823223702  …  0.008938790995803525, 0.008998371979429515, 0.008988836928898375, 0.008947978620468033, 0.008918411566546797, 0.008864222874619798, 0.008869869646660783, 0.008857306100568701, 0.008841079887577652, 0.008843759024369402], [0.00034193308437345017, 0.06037435247559266, 0.06909178597581046, 0.036237817335699855, 0.012076048471932168, 0.0058055174991

In [15]:
plot(sol.distance_trace)

data: [
  "scatter with fields type, x, and y"
]

layout: "layout with fields margin and template"

In [13]:
g(t) = ann([t], sol.params)[1]/2π
f1(t) = ann([t], sol.params)[2]/2π
f2(t) = ann([t], sol.params)[3]/2π

"g(t) = ann([t], θ)[1]/2π\nf1(t) = ann([t], θ)[2]/2π\nf2(t) = ann([t], θ)[3]/2π\n"

In [14]:
ts = collect(t0:t1/100:t1)
f= plot([
        scatter(x=ts, y=g.(ts), name="g")
        scatter(x=ts, y=f1.(ts), name="f₁")
        scatter(x=ts, y=f2.(ts), name="f₂")
    ],
    Layout(
        xaxis_title_text="Time (ns)",
        yaxis_title_text="Frequency (GHz)",
        legend=attr(x=1, y=1,),
        font=attr(
            size=16,
        )
    )
)


data: [
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y"
]

layout: "layout with fields font, legend, margin, template, xaxis, and yaxis"